<a href="https://colab.research.google.com/github/jon309/BIG_DATA/blob/main/big_data_Tp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import requests, time, random, csv, os, re
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm


BASE_URL = "https://dorar.net/hadith/sharh/{}"
HEADERS = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
                         "(KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}
START_ID = 1
MAX_ID = 10000
TARGET_COUNT = 1200
SAVE_EVERY = 50
CSV_PATH = "dorar_hadiths.csv"


session = requests.Session()
session.headers.update(HEADERS)

def clean_text(s):
    if not s:
        return ""
    s = re.sub(r'[\u200b\u200e\u200f]', '', s)
    s = re.sub(r'\s+\n', '\n', s)
    s = re.sub(r'\n{2,}', '\n', s)
    s = s.strip()
    return s

def safe_get(url, max_retries=4):
    backoff = 1.0
    for attempt in range(max_retries):
        try:
            resp = session.get(url, timeout=12)
            if resp.status_code == 200:
                return resp.text
            else:
                if 400 <= resp.status_code < 500:
                    return None
                time.sleep(backoff + random.random())
                backoff *= 2
        except requests.RequestException:
            time.sleep(backoff + random.random())
            backoff *= 2
    return None

def parse_dorar_page(html):
    soup = BeautifulSoup(html, "html.parser")

    hadith_elem = soup.select_one("div#cntnt article h5.h5-responsive") or soup.select_one("div#cntnt article h5")
    hadith_text = clean_text(hadith_elem.get_text(" ", strip=True)) if hadith_elem else ""

    strongs = soup.find_all("strong")

    def find_after_keyword(keyword):
        for s in strongs:
            txt = s.get_text(" ", strip=True)
            if keyword in txt:
                sp = s.find("span", class_="primary-text-color")
                if sp:
                    return clean_text(sp.get_text(" ", strip=True))
                nxt = s.find_next_sibling(text=True)
                if nxt:
                    return clean_text(nxt)
        return ""

    hukm = ""
    for s in strongs:
        txt = s.get_text(" ", strip=True)
        if "خلاصة حكم المحدث" in txt or "حكم المحدث" in txt:
            sp = s.find("span")
            if sp:
                hukm = clean_text(sp.get_text())
            else:
                m = re.search(r'\[.*?\]', txt)
                if m:
                    hukm = m.group(0)
            break

    rawi = find_after_keyword("الراوي") or ""
    source = find_after_keyword("المصدر") or ""
    page_num = find_after_keyword("الصفحة") or find_after_keyword("الرقم") or ""

    takhrij = ""
    for s in strongs:
        if "التخريج" in s.get_text(" ", strip=True):
            sp = s.find("span", class_="primary-text-color")
            if sp:
                takhrij = clean_text(sp.get_text(" ", strip=True))
            else:
                takhrij = clean_text(s.get_text(" ", strip=True))
            break

    categories = [clean_text(a.get_text(" ", strip=True)) for a in soup.select("a.badge-dorar")]
    categories_joined = "|".join(categories)

    return {
        "hadith": hadith_text,
        "hukm": hukm,
        "rawi": rawi,
        "source": source,
        "page": page_num,
        "takhrij": takhrij,
        "categories": categories_joined
    }

collected = []
start_from = START_ID
if os.path.exists(CSV_PATH):
    try:
        df_existing = pd.read_csv(CSV_PATH, dtype=str)
        if 'id' in df_existing.columns:
            last_id = int(df_existing['id'].astype(int).max())
            start_from = last_id + 1
        else:
            start_from = START_ID + len(df_existing)
        collected = df_existing.to_dict(orient="records")
        print(f"Resuming: loaded {len(collected)} rows from existing CSV. Continuing from id {start_from}.")
    except Exception as e:
        print("Could not load existing CSV to resume. Starting fresh. Error:", e)
        collected = []
        start_from = START_ID

print(f"Starting scraping from id {start_from} up to {MAX_ID} until {TARGET_COUNT} records collected.")
pbar = tqdm(total=TARGET_COUNT - len(collected), desc="Remaining")

i = start_from
saved_since = 0
while i <= MAX_ID and len(collected) < TARGET_COUNT:
    url = BASE_URL.format(i)
    html = safe_get(url)
    if html:
        data = parse_dorar_page(html)
        if data.get("hadith"):
            record = {"id": i, **data}
            collected.append(record)
            pbar.update(1)
            saved_since += 1
    else:
        pass

    time.sleep(random.uniform(0.5, 1.2))
    i += 1

    if saved_since >= SAVE_EVERY:
        df = pd.DataFrame(collected)
        df.to_csv(CSV_PATH, index=False, encoding="utf-8-sig")
        print(f"Saved {len(collected)} rows to {CSV_PATH} (checkpoint).")
        saved_since = 0

df_final = pd.DataFrame(collected)
df_final.to_csv(CSV_PATH, index=False, encoding="utf-8-sig")
print(f"Finished. Total rows collected: {len(collected)}")
print(f"CSV saved to: {CSV_PATH}")

df_final.head(5)

Starting scraping from id 1 up to 10000 until 1200 records collected.



Remaining: 100%|██████████| 10/10 [01:42<00:00, 10.30s/it]

Remaining:   4%|▍         | 50/1200 [01:25<28:34,  1.49s/it]

Saved 50 rows to dorar_hadiths.csv (checkpoint).



Remaining:   8%|▊         | 100/1200 [02:44<26:07,  1.42s/it]

Saved 100 rows to dorar_hadiths.csv (checkpoint).


...


Remaining:  96%|█████████▌| 1150/1200 [33:46<01:17,  1.56s/it]

Saved 1150 rows to dorar_hadiths.csv (checkpoint).



Remaining: 100%|██████████| 1200/1200 [35:23<00:00,  2.46s/it]

Saved 1200 rows to dorar_hadiths.csv (checkpoint).
Finished. Total rows collected: 1200
CSV saved to: dorar_hadiths.csv


,id,hadith,hukm,rawi,source,page,takhrij,categories
0,1,- دَخَلَ عَلَيَّ أَبُو بَكْرٍ وَعِندِي جَارِيَ...,[صحيح],عائشة أم المؤمنين,صحيح مسلم,892,أخرجه البخاري (952)، ومسلم (892).,حكم الأغاني - الغناء|حكم الأغاني - المعازف وال...
1,2,- دَخَلَ رَسولُ اللهِ صَلَّى اللَّهُ عليه وسلّ...,[صحيح],عائشة أم المؤمنين,صحيح مسلم,892,أخرجه البخاري (2906)، ومسلم (892).,آداب المجلس - ما يجوز من نظر النساء إلى الرجال...
2,3,- أنَّ أبَا بَكْرٍ، دَخَلَ عَلَيْهَا والنبيُّ...,[صحيح],عائشة أم المؤمنين,صحيح البخاري,3931,أخرجه الطبراني في ((المعجم الكبير)) (23/ 180) ...,حكم الأغاني - الغناء|حكم الأغاني - المعازف وال...
3,4,- دَخَلَ عَلَيَّ رَسولُ اللَّهِ صَلَّى اللهُ ...,[صحيح],عائشة أم المؤمنين,صحيح البخاري,2906,أخرجه البخاري (2906)، ومسلم (892),آداب المجلس - ما يجوز من نظر النساء إلى الرجال...
4,5,- أَخَذَ عُمَرُ جُبَّةً مِن إسْتَبْرَقٍ تُباع...,[صحيح],عبدالله بن عمر,صحيح البخاري,948,أخرجه البخاري (948)، ومسلم (2068),زينة اللباس - إباحة الذهب والحرير للنساء دون ا...
